# Multimethod polymorphism

## Polymorphism and its types

### Ad-hoc polymorphism

"closed-dispatch" polymorphism

In [5]:
; A function that ckecks the type of 'thing' argument 
; to call different functions based on the type

(defn ad-hoc-type-namer [thing]
    (condp = (type thing) 
        java.lang.String "string" 
        clojure.lang.PersistentVector "vector"))


#'user/ad-hoc-type-namer

In [2]:
; Passing a string parameter

(ad-hoc-type-namer "I'm a string")

"string"

In [3]:
; Passing a vector parameter

(ad-hoc-type-namer [])

"vector"

In [23]:
; If there’s a type this function doesn’t know how to handle, an exception is thrown

(ad-hoc-type-namer {}) 

Execution error (IllegalArgumentException) at user/ad-hoc-type-namer (REPL:3).
No matching clause: class clojure.lang.PersistentArrayMap


class java.lang.IllegalArgumentException: 

"open-dispatch" polymorphism

In [16]:
; Pull type implementations out into a separate map

(def type-namer-implementations 
    {java.lang.String (fn [thing] "string")
     clojure.lang.PersistentVector (fn [thing] "vector")})


#'user/type-namer-implementations

In [17]:
; A function that ckecks the type of 'thing' argument 
; to call different functions based on the type, 
; without a fixed set of types to ckeck
 
(defn open-ad-hoc-type-namer [thing]
    (let [dispatch-value (type thing)] 
        ; Use a dispatch value as key to implementation map
        (if-let [implementation (get type-namer-implementations dispatch-value)] 
            (implementation thing) 
            (throw (IllegalArgumentException. (str "No implementation found for " dispatch-value))))))


#'user/open-ad-hoc-type-namer

In [18]:
; Passing a parameter of string type

(open-ad-hoc-type-namer "I'm a string") 

"string"

In [24]:
; Passing a parameter of vector type

(open-ad-hoc-type-namer []) 

"vector"

In [20]:
; Passing a parameter of map type (it will throw an exception)

(open-ad-hoc-type-namer {}) 

Execution error (IllegalArgumentException) at user/open-ad-hoc-type-namer (REPL:10).
No implementation found for class clojure.lang.PersistentArrayMap


class java.lang.IllegalArgumentException: 

In [21]:
; Adding a 'map' type to a the map of types

(def type-namer-implementations 
    (assoc type-namer-implementations clojure.lang.PersistentArrayMap (fn [thing] "map")))


#'user/type-namer-implementations

In [22]:
; Passing a parameter of map type again (it won't throw an exception now)

(open-ad-hoc-type-namer {})

"map"

### Subtype polymorphism

Checking types with ad-hoc polymorphism

In [25]:
; A function that checks if its argument is map-like

(defn map-type-namer [thing] 
    (condp = (type thing)
        clojure.lang.PersistentArrayMap "map"
        clojure.lang.PersistentHashMap "map")) ; Notice the code duplication

#'user/map-type-namer

In [26]:
; Passing a parameter of hash-map type (recognized)

(map-type-namer (hash-map))

"map"

In [27]:
; Passing a parameter of array-map type (recognized)

(map-type-namer (array-map))

"map"

In [28]:
; Passing a parameter of sorted-map type (unrecognized)

(map-type-namer (sorted-map)) 

Execution error (IllegalArgumentException) at user/map-type-namer (REPL:2).
No matching clause: class clojure.lang.PersistentTreeMap


class java.lang.IllegalArgumentException: 

Checking types with subtype polymorphism

In [33]:
(defn subtyping-map-type-namer [thing] 
    (cond
        (instance? clojure.lang.APersistentMap thing) "map" ; APersistent-Map is Java superclass of all map-like things in Clojure
        :else (throw (IllegalArgumentException. (str "No implementation found for ") (type thing)))))


#'user/subtyping-map-type-namer

In [30]:
; Passing a parameter of hash-map type (recognized)

(subtyping-map-type-namer (hash-map))

"map"

In [31]:
; Passing a parameter of array-map type (recognized)

(subtyping-map-type-namer (array-map))

"map"

In [32]:
; Passing a parameter of sorted-map type (recognized now)

(subtyping-map-type-namer (sorted-map)) 

"map"